<a href="https://colab.research.google.com/github/Duyih25/Bomberman/blob/main/dNDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/churn/sub_model_t5.csv')

data.drop('ISDN', axis= 1, inplace= True)

data.drop('TD_5', axis= 1, inplace= True)
data.drop('DATA_5', axis= 1, inplace= True)
data.drop('ONNET_IN_5', axis= 1, inplace= True)
data.drop('ONNET_OUT_5', axis= 1, inplace= True)
data.drop('OFFNET_IN_5', axis= 1, inplace= True)
data.drop('OFFNET_OUT_5', axis= 1, inplace= True)
data.drop('PACK_TIME_5', axis= 1, inplace= True)
data.drop('PACK_MONEY_5', axis= 1, inplace= True)
data.drop('RC_TIME_5', axis= 1, inplace= True)
data.drop('RC_MONEY_5', axis= 1, inplace= True)
data.drop('REG_ON_5', axis= 1, inplace= True)

# encode data
data = pd.get_dummies(data)

data['15c3d_5'] = data['15c3d_5'].astype(str)
data['15c3d_5'].replace(["0", "1"], ["y", "n"], inplace= True)

data.dtypes

PRODUCT_CODE      int64
AGE               int64
PROVINCE          int64
TD_2            float64
TD_3            float64
TD_4            float64
DATA_2          float64
DATA_3          float64
DATA_4          float64
ONNET_IN_2      float64
ONNET_IN_3      float64
ONNET_IN_4      float64
ONNET_OUT_2     float64
ONNET_OUT_3     float64
ONNET_OUT_4     float64
OFFNET_IN_2     float64
OFFNET_IN_3     float64
OFFNET_IN_4     float64
OFFNET_OUT_2    float64
OFFNET_OUT_3    float64
OFFNET_OUT_4    float64
PACK_TIME_2       int64
PACK_TIME_3       int64
PACK_TIME_4       int64
PACK_MONEY_2    float64
PACK_MONEY_3    float64
PACK_MONEY_4    float64
RC_TIME_2         int64
RC_TIME_3         int64
RC_TIME_4         int64
RC_MONEY_2        int64
RC_MONEY_3        int64
RC_MONEY_4        int64
REG_ON_2          int64
REG_ON_3          int64
REG_ON_4          int64
15c3d_2           int64
15c3d_3           int64
15c3d_4           int64
15c3d_5          object
OS_FF             uint8
OS_SM           

In [ ]:
### finding ouliers
def outliers(df, ft):
    if ft in ("ISDN","PRODUCT_CODE","AGE","PROVINCE","OS","15c3d_5"):
        return []
    
    Q1 = df[ft].quantile(0.05)
    Q3 = df[ft].quantile(0.95)
    IQR = Q3 - Q1
    
    upper_bound = Q3 + 20 * IQR
    return df.index[(df[ft] < 0) | (df[ft] > upper_bound)].tolist()

In [ ]:
index_list = []

for col in data:
    index_list.extend(outliers(data, col))

index_list = list(dict.fromkeys(index_list))    
print(len(index_list))
    

127


In [ ]:
# remove outliers

def remove(df, ls):
    ls = sorted(ls)
    df = df.drop(ls)
    return df

In [ ]:
data = remove(data, index_list)

In [ ]:
data_train,data_test = train_test_split(data, test_size= 0.2, random_state= 42)

In [ ]:
print(data_train.head())
print(data_test.head())

       PRODUCT_CODE  AGE  PROVINCE     TD_2  ...  15c3d_5  OS_FF  OS_SM  OS_Unknow
72248      10000022  132  10000015   0.0000  ...        n      0      1          0
10124      10000001  932  10000016   3.5268  ...        y      0      1          0
89681      10000001  202  10000009   0.1330  ...        n      1      0          0
70388      10000013  187  10000008  10.0000  ...        y      1      0          0
77474      10000001  369  10000001  40.0000  ...        n      0      0          1

[5 rows x 43 columns]
       PRODUCT_CODE  AGE  PROVINCE     TD_2  ...  15c3d_5  OS_FF  OS_SM  OS_Unknow
36228      10000001   94  10000018   0.0000  ...        n      0      1          0
61537      10000001  762  10000025  10.0000  ...        y      0      1          0
46150      10000001  797  10000003  21.4303  ...        n      1      0          0
13581      10000001  264  10000001   0.0000  ...        y      0      1          0
77044      10000001  117  10000015   0.0000  ...        n      0

In [ ]:
data_train.to_csv("train.csv", index=False, header=False)
data_test.to_csv("test.csv", index=False, header=False)

In [ ]:
# A list of the numerical feature names.
NUMERIC_FEATURE_NAMES = data.drop("15c3d_5", axis=1).columns
# A dictionary of the categorical features and their vocabulary.
CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    
}
# A list of the columns to ignore from the dataset.
#IGNORE_COLUMN_NAMES = ["fnlwgt"]
# A list of the categorical feature names.
CATEGORICAL_FEATURE_NAMES = []
# A list of all the input features.
FEATURE_NAMES = NUMERIC_FEATURE_NAMES
# A list of column default values for each feature.
COLUMN_DEFAULTS = [
    [0.0] if feature_name in NUMERIC_FEATURE_NAMES else ["NA"]
    for feature_name in data.columns
]
# The name of the target feature.
TARGET_FEATURE_NAME = "15c3d_5"
# A list of the labels of the target features.
TARGET_LABELS = ["y", "n"]




In [ ]:
from tensorflow.keras.layers import StringLookup

target_label_lookup = StringLookup(
    vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
)


def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=data.columns,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=False,
        na_value=" ",
        shuffle=shuffle,
    ).map(lambda features, target: (features, target_label_lookup(target)))
    return dataset.cache()

In [ ]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return inputs

In [ ]:
def encode_inputs(inputs):
    encoded_features = []
    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            # Create a lookup to convert a string values to an integer indices.
            # Since we are not using a mask token, nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and num_oov_indices to 0.
            lookup = StringLookup(
                vocabulary=vocabulary, mask_token=None, num_oov_indices=0
            )
            # Convert the string input values into integer indices.
            value_index = lookup(inputs[feature_name])
            embedding_dims = int(math.sqrt(lookup.vocabulary_size()))
            # Create an embedding layer with the specified dimensions.
            embedding = layers.Embedding(
                input_dim=lookup.vocabulary_size(), output_dim=embedding_dims
            )
            # Convert the index values to embedding representations.
            encoded_feature = embedding(value_index)
        else:
            # Use the numerical features as-is.
            encoded_feature = inputs[feature_name]
            if inputs[feature_name].shape[-1] is None:
                encoded_feature = tf.expand_dims(encoded_feature, -1)

        encoded_features.append(encoded_feature)

    encoded_features = layers.concatenate(encoded_features)
    return encoded_features

In [ ]:
class NeuralDecisionTree(keras.Model):
    def __init__(self, depth, num_features, used_features_rate, num_classes):
        super(NeuralDecisionTree, self).__init__()
        self.depth = depth
        self.num_leaves = 2 ** depth
        self.num_classes = num_classes

        # Create a mask for the randomly selected features.
        num_used_features = int(num_features * used_features_rate)
        one_hot = np.eye(num_features)
        sampled_feature_indicies = np.random.choice(
            np.arange(num_features), num_used_features, replace=False
        )
        self.used_features_mask = one_hot[sampled_feature_indicies]

        # Initialize the weights of the classes in leaves.
        self.pi = tf.Variable(
            initial_value=tf.random_normal_initializer()(
                shape=[self.num_leaves, self.num_classes]
            ),
            dtype="float32",
            trainable=True,
        )

        # Initialize the stochastic routing layer.
        self.decision_fn = layers.Dense(
            units=self.num_leaves, activation="sigmoid", name="decision"
        )

    def call(self, features):
        batch_size = tf.shape(features)[0]

        # Apply the feature mask to the input features.
        features = tf.matmul(
            features, self.used_features_mask, transpose_b=True
        )  # [batch_size, num_used_features]
        # Compute the routing probabilities.
        decisions = tf.expand_dims(
            self.decision_fn(features), axis=2
        )  # [batch_size, num_leaves, 1]
        # Concatenate the routing probabilities with their complements.
        decisions = layers.concatenate(
            [decisions, 1 - decisions], axis=2
        )  # [batch_size, num_leaves, 2]

        mu = tf.ones([batch_size, 1, 1])

        begin_idx = 1
        end_idx = 2
        # Traverse the tree in breadth-first order.
        for level in range(self.depth):
            mu = tf.reshape(mu, [batch_size, -1, 1])  # [batch_size, 2 ** level, 1]
            mu = tf.tile(mu, (1, 1, 2))  # [batch_size, 2 ** level, 2]
            level_decisions = decisions[
                :, begin_idx:end_idx, :
            ]  # [batch_size, 2 ** level, 2]
            mu = mu * level_decisions  # [batch_size, 2**level, 2]
            begin_idx = end_idx
            end_idx = begin_idx + 2 ** (level + 1)

        mu = tf.reshape(mu, [batch_size, self.num_leaves])  # [batch_size, num_leaves]
        probabilities = keras.activations.softmax(self.pi)  # [num_leaves, num_classes]
        outputs = tf.matmul(mu, probabilities)  # [batch_size, num_classes]
        return outputs

In [ ]:
class NeuralDecisionForest(keras.Model):
    def __init__(self, num_trees, depth, num_features, used_features_rate, num_classes):
        super(NeuralDecisionForest, self).__init__()
        self.ensemble = []
        # Initialize the ensemble by adding NeuralDecisionTree instances.
        # Each tree will have its own randomly selected input features to use.
        for _ in range(num_trees):
            self.ensemble.append(
                NeuralDecisionTree(depth, num_features, used_features_rate, num_classes)
            )

    def call(self, inputs):
        # Initialize the outputs: a [batch_size, num_classes] matrix of zeros.
        batch_size = tf.shape(inputs)[0]
        outputs = tf.zeros([batch_size, num_classes])

        # Aggregate the outputs of trees in the ensemble.
        for tree in self.ensemble:
            outputs += tree(inputs)
        # Divide the outputs by the ensemble size to get the average.
        outputs /= len(self.ensemble)
        return outputs

In [ ]:
learning_rate = 0.01
batch_size = 265
num_epochs = 10
hidden_units = [64, 64]


def run_experiment(model):

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    print("Start training the model...")
    train_dataset = get_dataset_from_csv(
        "train.csv", shuffle=True, batch_size=batch_size
    )

    model.fit(train_dataset, epochs=num_epochs)
    print("Model training finished")

    print("Evaluating the model on the test data...")
    test_dataset = get_dataset_from_csv("test.csv", batch_size=batch_size)

    _, accuracy = model.evaluate(test_dataset)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

In [ ]:
num_trees = 10
depth = 10
used_features_rate = 1.0
num_classes = len(TARGET_LABELS)


def create_tree_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs)
    features = layers.BatchNormalization()(features)
    num_features = features.shape[1]

    tree = NeuralDecisionTree(depth, num_features, used_features_rate, num_classes)

    outputs = tree(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


tree_model = create_tree_model()
run_experiment(tree_model)

Start training the model...
Epoch 1/10
302/302 [==============================] - 14s 40ms/step - loss: 0.5059 - sparse_categorical_accuracy: 0.7618
Epoch 2/10
302/302 [==============================] - 10s 33ms/step - loss: 0.4745 - sparse_categorical_accuracy: 0.7725
Epoch 3/10
302/302 [==============================] - 10s 32ms/step - loss: 0.4702 - sparse_categorical_accuracy: 0.7744
Epoch 4/10
302/302 [==============================] - 10s 33ms/step - loss: 0.4672 - sparse_categorical_accuracy: 0.7767
Epoch 5/10
302/302 [==============================] - 10s 33ms/step - loss: 0.4646 - sparse_categorical_accuracy: 0.7777
Epoch 6/10
302/302 [==============================] - 10s 33ms/step - loss: 0.4620 - sparse_categorical_accuracy: 0.7794
Epoch 7/10
302/302 [==============================] - 10s 33ms/step - loss: 0.4592 - sparse_categorical_accuracy: 0.7822
Epoch 8/10
302/302 [==============================] - 11s 35ms/step - loss: 0.4563 - sparse_categorical_accuracy: 0.7849
Epoc

In [ ]:
num_trees = 25
depth = 10
used_features_rate = 0.5


def create_forest_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs)
    features = layers.BatchNormalization()(features)
    num_features = features.shape[1]

    forest_model = NeuralDecisionForest(
        num_trees, depth, num_features, used_features_rate, num_classes
    )

    outputs = forest_model(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


forest_model = create_forest_model()

run_experiment(forest_model)

Start training the model...
Epoch 1/10
302/302 [==============================] - 202s 588ms/step - loss: 0.5109 - sparse_categorical_accuracy: 0.7516
Epoch 2/10
154/302 [==============>...............] - ETA: 1:27 - loss: 0.4773 - sparse_categorical_accuracy: 0.7702